## 1. Framing the Problem

The objective of this lab is to perform univariate linear regression on the California Housing Prices dataset. We'll predict the median house value based on three individual features: median income, population, and number of households. Our goal is to identify which feature offers the best predictive power for median house value using linear regression evaluation metrics.


### Import Packages

In [ ]:
import requests
import zipfile
import pandas as pd
import os

Dataset Source: [California Housing Prices](https://www.kaggle.com/api/v1/datasets/download/camnugent/california-housing-prices)


In [ ]:


# --- Configuration ---
zip_url = "https://www.kaggle.com/api/v1/datasets/download/camnugent/california-housing-prices"
project_data_dir = os.path.join(".", "data")
zip_file_name = "california-housing-prices.zip"
zip_file_path = os.path.join(project_data_dir, zip_file_name)
extract_dir = os.path.join(project_data_dir, "california_housing_data")
cleanup_zip = True

# --- Ensure directories exist ---
os.makedirs(project_data_dir, exist_ok=True)
os.makedirs(extract_dir, exist_ok=True)

# --- Download ZIP from Kaggle (will fail without auth) ---
try:
    print("Downloading dataset from Kaggle...")
    response = requests.get(zip_url, stream=True)
    response.raise_for_status() 

    with open(zip_file_path, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print("Download completed.")

except requests.exceptions.RequestException as e:
    print(f"Download failed: {e}")
    exit(1)

# --- Extract CSV file ---
try:
    print("Extracting dataset...")
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

    # Find CSV file
    csv_file_name = next((f for f in os.listdir(extract_dir) if f.endswith('.csv')), None)
    if not csv_file_name:
        raise FileNotFoundError("No CSV file found in the extracted content.")

    # Load into DataFrame
    csv_file_path = os.path.join(extract_dir, csv_file_name)
    df = pd.read_csv(csv_file_path)

    print(df.head())
    print("\n DataFrame Info:")
    df.info()

except zipfile.BadZipFile:
    print("Error: Downloaded file is not a valid ZIP file.")
except Exception as e:
    print(f"Error during extraction or reading: {e}")

# --- Clean up ZIP file ---
if cleanup_zip and os.path.exists(zip_file_path):
    os.remove(zip_file_path)


Download completed.
Extracting dataset...
   longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0    -122.23     37.88                41.0        880.0           129.0   
1    -122.22     37.86                21.0       7099.0          1106.0   
2    -122.24     37.85                52.0       1467.0           190.0   
3    -122.25     37.85                52.0       1274.0           235.0   
4    -122.25     37.85                52.0       1627.0           280.0   

   population  households  median_income  median_house_value ocean_proximity  
0       322.0       126.0         8.3252            452600.0        NEAR BAY  
1      2401.0      1138.0         8.3014            358500.0        NEAR BAY  
2       496.0       177.0         7.2574            352100.0        NEAR BAY  
3       558.0       219.0         5.6431            341300.0        NEAR BAY  
4       565.0       259.0         3.8462            342200.0        NEAR BAY  

 DataFrame Info:
<class 'pandas.